<h1 align="center">W203 Lab 3: Covid 19</h1>
<h4 align="center">Blake Allen, Sam Shih, Mike Fan</h4>
<h4 align="center">Summer 2020 Section 1 </h4>

# 1. Introduction

The dataset includes (1) variables representing the spread of the disease, (2) variables representing state-level policy responses, and (3) general state-level characteristics.

## Research Question of Interest

1. Which response policy has been the most effective in reducing COVID case count (or death count)?
    - **Dependent variable:** Case count (or death count)
    - **Potential predictors:** # of days between state of emergency and shelter in place (SIP); # of days between SIP and end of SIP; # of days between state of emergency and non-essential business closure; # of days between non-essential business closure and business re-opening; # of days between face mask mandate by employees of public-facing employees and business re-opening. Other contributing predicators could be population density per square mile, total_test, % living under the federal poverty line, % at risk of serious illness due to COVID, % of state population between age 19 - 54 . If we want to examine death, then we can look at fatality specific metrics, which some we need to derive ourselves: death_100k (death / 100k), fatality rate (total death / total cases), All cause deaths 2018, and % of state population between age 55+
2. What are the key factors that contribute to increases in COVID case count?
    - **Dependent variable:** Case count
    - **Potential predictors:** Identical to those in question 1 except fatality specific variables
3. What are the key factors that contribute to higher fatality rate of COVID 19?
    - **Dependent variable:** Fatality rate
    - **Potential predictors:** Identical to those in question 1 minus the case count specific ones

#### Notes
- Data was pulled on July 5, 2020 (corroborated totalTestResults with Alabama's [historical data](https://covidtracking.com/data/state/alabama#historical) that's tracked and made available by [The COVID Tracking Project](https://covidtracking.com/)
- 2 records of Arizona (one with a capital A and the other with lower case a) appears in the dataset. All variable values are the same except for total cases and total death. Checking the data against CDC, it seems like we should combine the total cases and total death value to form a single record for the state of Arizona
- Kentucky's totalTestResults is missing, but it's available [covidtracking.com](https://covidtracking.com/data/state/kentucky#historical), which is 395,647 cumulative at the end of July 5th, 2020

# 2. Exploratory Data Analysis (EDA)

## Data Review

In [12]:
library(plyr)
library(dplyr)
library(ggplot2)
library(summarytools)
library(coin)
library(effsize)
library(tidyverse)
library(lsr)
library(corrplot)
library(rstatix)
library(BSDA)
library(cowplot)
library(ggrepel)
library(stargazer)
library(knitr)
library(kableExtra)
library(corrr)
library(ggcorrplot)
library(ggfortify)
library(lmtest)

In [11]:
df_covid = read.csv("covid-19_dist0720.csv")

head(df_covid)
summary(df_covid)
str(df_covid)

,State,Total.Cases,Total.Death,Death_100k,CasesInLast7Days,RatePer100000,totalTestResults,State.of.emergency,Stay.at.home..shelter.in.place,End.relax.stay.at.home.shelter.in.place,...,Population.2018,Percent.living.under.the.federal.poverty.line..2018.,Percent.at.risk.for.serious.illness.due.to.COVID,All.cause.deaths.2018,Children.0.18,Adults.19.25,Adults.26.34,Adults.35.54,Adults.55.64,X65.
,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<chr>,<chr>,<chr>,...,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alabama,44909,1009,20.6,9804,918.8,449886,3/13/2020,4/4/2020,4/30/2020,...,4887871,16.8,43.1,54352,0.24,0.09,0.12,0.25,0.14,0.17
2,Alaska,1138,16,2.2,284,154.3,122732,3/11/2020,3/28/2020,4/24/2020,...,737438,10.9,32.8,4453,0.27,0.09,0.13,0.26,0.13,0.12
3,Arizona,83376,1538,25.2,28038,1367.7,604362,3/11/2020,3/31/2020,5/16/2020,...,7171646,14.0,39.1,59282,0.24,0.09,0.12,0.24,0.12,0.18
4,arizona,14713,271,25.2,28038,1367.7,604362,3/11/2020,3/31/2020,5/16/2020,...,7171646,14.0,39.1,59282,0.24,0.09,0.12,0.24,0.12,0.18
5,Arkansas,23814,287,9.5,4504,790.2,338893,3/11/2020,0,0,...,3013825,17.2,43.5,32336,0.25,0.09,0.12,0.25,0.13,0.17
6,California,260155,6331,16.0,53722,657.7,4680138,3/4/2020,3/19/2020,0,...,39557045,12.8,33.3,268818,0.24,0.09,0.14,0.26,0.12,0.14


    State            Total.Cases      Total.Death        Death_100k    
 Length:52          Min.   :   939   Min.   :   16.0   Min.   :  1.30  
 Class :character   1st Qu.: 11882   1st Qu.:  275.5   1st Qu.:  9.50  
 Mode  :character   Median : 31305   Median :  811.5   Median : 18.20  
                    Mean   : 55332   Mean   : 2493.1   Mean   : 41.99  
                    3rd Qu.: 67053   3rd Qu.: 2734.8   3rd Qu.: 43.33  
                    Max.   :398828   Max.   :31906.0   Max.   :354.30  
                                                                       
 CasesInLast7Days  RatePer100000    totalTestResults  State.of.emergency
 Min.   :   51.0   Min.   :  66.1   Min.   :  35327   Length:52         
 1st Qu.:  959.5   1st Qu.: 467.5   1st Qu.: 187374   Class :character  
 Median : 3218.5   Median : 723.9   Median : 419509   Mode  :character  
 Mean   : 7858.6   Mean   : 837.3   Mean   : 699739                     
 3rd Qu.: 6113.2   3rd Qu.:1036.4   3rd Qu.: 813577        

'data.frame':	52 obs. of  25 variables:
 $ State                                                         : chr  "Alabama" "Alaska" "Arizona" "arizona" ...
 $ Total.Cases                                                   : int  44909 1138 83376 14713 23814 260155 34065 46717 12348 10482 ...
 $ Total.Death                                                   : int  1009 16 1538 271 287 6331 1701 4335 512 559 ...
 $ Death_100k                                                    : num  20.6 2.2 25.2 25.2 9.5 ...
 $ CasesInLast7Days                                              : int  9804 284 28038 28038 4504 53722 2043 511 1095 266 ...
 $ RatePer100000                                                 : num  919 154 1368 1368 790 ...
 $ totalTestResults                                              : int  449886 122732 604362 604362 338893 4680138 350717 499669 123941 105993 ...
 $ State.of.emergency                                            : chr  "3/13/2020" "3/11/2020" "3/11/2020" "3/11/2020"

## Data Cleansing

# 3. Modeling Process

## Overview

## Dependent Variable

## Model 1

### Independent Variable Selection

### Model Execution and Interpretation

### Model Diagnostics

## Model 2

### Independent Variable Selection

### Model Execution and Interpretation

### Model Diagnostics

## Model 3

### Independent Variable Selection

### Model Execution and Interpretation

### Model Diagnostics

# 4. Assessment of the CLM Assumptions

## MLR.1: Linearity in Parameters

## MLR.2: Random Sampling

## MLR.3: No Perfect Collinearity

## MLR.4: Zero Conditional Mean

## MLR.5: Homoskedasticity

## MLR.6: Normality of Errors

# 5. Regression Table

### Statistical Significance

### Practical Significance

# 6. Omitted Variables Discussion

# 7. Conclusion